In [1]:
import statistics
import pandas as pd
import numpy as np
from tqdm import tqdm
from collections import Counter
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
import matplotx

pd.set_option('display.max_columns', None)

In [2]:
df = pd.read_json('dataset/1_mil_final_incas_format.jsonl', lines=True)
df

,id,name,title,contentText,translatedTitle,translatedContentText,annotations,mediaType,mentionedUsers,author,mediaTypeAttributes,timePublished,language,embeddedUrls,imageUrls
0,0000e13c70f29460db129891d484d9c5b9384b6f,messages/0000e13c70f29460db129891d484d9c5b9384b6f,RT @dupontaignan La honte pour la France ! Un ...,RT @dupontaignan La honte pour la France ! Un ...,None,None,[],Twitter,[],None,"{'twitterData': {'retweetCount': 0, 'twitterAu...",1653772228000,French,[],[]
1,00002e64a617ef224d125e2acdc533ce65f3652e,messages/00002e64a617ef224d125e2acdc533ce65f3652e,RT @JLMelenchon Le service politique de France...,RT @JLMelenchon Le service politique de France...,None,None,[],Twitter,[],None,"{'twitterData': {'retweetCount': 0, 'twitterAu...",1655065494000,French,[],[]
2,0004d7991f07b279cc6dfcbb667065b7e57b4196,messages/0004d7991f07b279cc6dfcbb667065b7e57b4196,RT @JLMelenchon Nous n'allons pas remâcher le ...,RT @JLMelenchon Nous n'allons pas remâcher le ...,None,None,[],Twitter,[],None,"{'twitterData': {'retweetCount': 0, 'twitterAu...",1651935781000,French,[],[http://pbs.twimg.com/media/FSKflROXEAImriv.jpg]
3,00042c86fa3c5cfd720d7294295981ffb96e85b0,messages/00042c86fa3c5cfd720d7294295981ffb96e85b0,"RT @EmmanuelMacron Bienvenue à Paris, cher @al...","RT @EmmanuelMacron Bienvenue à Paris, cher @al...",None,None,[],Twitter,[79dd7b8528f807dfbfedaafadde5ce03],None,"{'twitterData': {'retweetCount': 0, 'twitterAu...",1652462635000,French,[],[http://pbs.twimg.com/media/FSpLKUSX0AQ8Isf.jpg]
4,00036a65fd97d3ae809d2ee24b7e968389e0f700,messages/00036a65fd97d3ae809d2ee24b7e968389e0f700,RT @UPR_Asselineau ⚠️ACCORD SECRET LREM-LR-RN ...,RT @UPR_Asselineau ⚠️ACCORD SECRET LREM-LR-RN ...,None,None,[],Twitter,[],None,"{'twitterData': {'retweetCount': 0, 'twitterAu...",1656539834000,French,[https://www.20minutes.fr/politique/3317471-20...,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1153524,ff8c289c533513e291b318f1deab5ee45ec896ec,messages/ff8c289c533513e291b318f1deab5ee45ec896ec,LA RUSSIE prévient l’Union Européenne : « pas ...,[LA RUSSIE prévient l’Union Européenne : « pas...,RUSSIA warns the European Union: “no payment i...,[RUSSIA warns the European Union: “no payment ...,"[{'id': 'c676176ab9bc6132d37a576a1813a2b2', 'n...",News,[],El Diablo,{},1648695840000,French,[],[https://image.over-blog.com/Fv0-ruiKPpfxPDfsI...
1153525,ff5a73b46f4f7a8d6613ca0ebd75f785b7e8d903,messages/ff5a73b46f4f7a8d6613ca0ebd75f785b7e8d903,RT @JLMelenchon 🔴 Rendez-vous ce jeudi 17 févr...,RT @JLMelenchon 🔴 Rendez-vous ce jeudi 17 févr...,"RT @JLMelenchon 🔴 See you this Thursday, Febru...","RT @JLMelenchon 🔴 See you this Thursday, Febru...",[],Twitter,"[cb9738bb0d492549c2959afff3391cd5, 3918e2b8115...",None,"{'twitterData': {'retweetCount': 0, 'twitterAu...",1644950102000,French,[],[https://video.twimg.com/amplify_video/1493634...
1153526,ffd08523e30ddf5dc81238b25e43e8b5d40147ca,messages/ffd08523e30ddf5dc81238b25e43e8b5d40147ca,RT @JLMelenchon Je vous invite à vous joindre ...,RT @JLMelenchon Je vous invite à vous joindre ...,RT @JLMelenchon I invite you to join in an eff...,RT @JLMelenchon I invite you to join in an eff...,[],Twitter,[],None,"{'twitterData': {'retweetCount': 0, 'twitterAu...",1644959037000,French,[https://www.youtube.com/watch?v=sT6qeNbmHB8&f...,[http://pbs.twimg.com/media/FLqMn4ZWQAMcGB5.jpg]
1153527,ff2128fb8fe913ac20d05d5bfbed0c7b521e28f6,messages/ff2128fb8fe913ac20d05d5bfbed0c7b521e28f6,Futur président. [ Zemmour ] 4458 - Politique,Xamoth a écrit : Terme inventé par Impacilla C...,future president. [ Zemmour ] 4458 - Politics,Xamoth wrote: Term invented by Impacilla Carpi...,"[{'id': '22a511d79824a0ee1313ff50d54cff8f', 'n...",Forum,[],l'helleno-​chretien,{},1648583040000,French,[],[]


In [3]:
df['datetime'] = [datetime.fromtimestamp(time/1000) for time in df['timePublished'].values]
dates = [pd.to_datetime(dt).date() for dt in df['datetime'].values]
df['date'] = dates
df = df.sort_values(by='date')
df

,id,name,title,contentText,translatedTitle,translatedContentText,annotations,mediaType,mentionedUsers,author,mediaTypeAttributes,timePublished,language,embeddedUrls,imageUrls,datetime,date
1143017,ea2ed2beb8e255ed88f004efbe0e3676513f371d,messages/ea2ed2beb8e255ed88f004efbe0e3676513f371d,RT @JLMelenchon Il y a des gens qui ne sont pa...,RT @JLMelenchon Il y a des gens qui ne sont pa...,RT @JLMelenchon There are people who are not h...,RT @JLMelenchon There are people who are not h...,[],Twitter,[cb9738bb0d492549c2959afff3391cd5],None,"{'twitterData': {'retweetCount': 0, 'twitterAu...",1488324082000,French,[],[],2017-02-28 15:21:22,2017-02-28
1089969,79ef49b00a0305877328be30bbc0831491c5b3b9,messages/79ef49b00a0305877328be30bbc0831491c5b3b9,Les demandes d’asile retrouvent le niveau d’av...,Ce SUV de 2700 ch atteint une vitesse de point...,Asylum applications return to pre-pandemic level,This 2700 hp SUV reaches a top speed of 355 km...,"[{'id': '0d3db5ba95f248b0ff4c117c08ac7547', 'n...",News,[],msn.com,{},1642291200000,French,"[http://www.24heures.ch/, https://www.24heures...",[https://img-s-msn-com.akamaized.net/tenant/am...,2022-01-15 16:00:00,2022-01-15
1090431,7beb2f4e1ef79d4f82c586726a77e0b7cbecb01e,messages/7beb2f4e1ef79d4f82c586726a77e0b7cbecb01e,Le gouvernement du Bénin répond aux «pressions...,Interpellé mercredi 12 janvier par un député d...,The government of Benin responds to &quot;pres...,Questioned on Wednesday January 12 by a member...,"[{'id': 'fd0e556a27d67648b9a2b5634549569e', 'n...",News,[],RFI,{},1642264380000,French,[],[https://ic-cdn.flipboard.com/rfi.fr/e3ab4537d...,2022-01-15 08:33:00,2022-01-15
1048319,222d917f1bf3f666f2b31b005c2c83c9f0af2adc,messages/222d917f1bf3f666f2b31b005c2c83c9f0af2adc,Cyberattaque: l'Ukraine accuse la Russie et di...,Ajoute déclarations du porte-parole du Kremlin...,Cyberattack: Ukraine accuses Russia and says i...,"Adds statements by Kremlin spokesperson, Biden...","[{'id': 'c388a9f428bba9ee64f9a3856ce8bced', 'n...",News,[],zonebourse.com,{},1642350660000,French,[https://gateway.mdgms.com/extern/logo_image.h...,[https://gateway.mdgms.com/extern/logo_image.h...,2022-01-16 08:31:00,2022-01-16
1044040,1986efa834e0be219905cf4ceab726203b7dbe28,messages/1986efa834e0be219905cf4ceab726203b7dbe28,Plus de 360 millions de chrétiens persécutés d...,Plus de 360 millions de chrétiens persécutés d...,More than 360 million Christians persecuted in...,More than 360 million Christians persecuted in...,"[{'id': '0009421ef61214966aec82b64b2b5b89', 'n...",News,[],None,{},1642550400000,French,[https://portesouvertes.fr/persecution-des-chr...,[https://img.20mn.fr/uhmowooDRvCYTuLAUMjQSw/64...,2022-01-18 16:00:00,2022-01-18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140565,034d90ee2908a8c5acbeaa575131fbdf697ededc,messages/034d90ee2908a8c5acbeaa575131fbdf697ededc,Politique. Vers une coalition des vaincus auto...,On sait maintenant ce que le président Macron ...,None,None,[],Blog,[],None,{},1656603938000,French,[],[],2022-06-30 08:45:38,2022-06-30
626701,0ec9939120b893de4b07757516e9f42f49b02f78,messages/0ec9939120b893de4b07757516e9f42f49b02f78,RT @EmmanuelMacron La Russie ne doit pas l’emp...,RT @EmmanuelMacron La Russie ne doit pas l’emp...,None,None,[],Twitter,[],None,"{'twitterData': {'retweetCount': 0, 'twitterAu...",1656594931000,French,[],[https://video.twimg.com/amplify_video/1542481...,2022-06-30 06:15:31,2022-06-30
843002,13e59181f4bb89db955c21488ac6171d57f6cb56,messages/13e59181f4bb89db955c21488ac6171d57f6cb56,"RT @vpecresse À Kyiv, rencontre avec le premie...","RT @vpecresse À Kyiv, rencontre avec le premie...",None,None,[],Twitter,[5cc362fe10fbc8c1b400b973b1ad56c0],None,"{'twitterData': {'retweetCount': 0, 'twitterAu...",1656592992000,French,[],[http://pbs.twimg.com/media/FWgEJlnWAAAHje1.jp...,2022-06-30 05:43:12,2022-06-30
645219,0f3e18efbb92900ba2108015ef1280512e90e153,messages/0f3e18efbb92900ba2108015ef1280512e90e153,« Seychelles » : différence entre les versions..

In [4]:
df = df[df["date"] != np.datetime64('2017-02-28')]

In [5]:
date_list = df["date"].tolist()
uni_date_list = list(set(date_list))
uni_date_list.sort()
uni_date_list

[datetime.date(2022, 1, 15),
 datetime.date(2022, 1, 16),
 datetime.date(2022, 1, 18),
 datetime.date(2022, 1, 19),
 datetime.date(2022, 1, 24),
 datetime.date(2022, 1, 28),
 datetime.date(2022, 1, 30),
 datetime.date(2022, 1, 31),
 datetime.date(2022, 2, 1),
 datetime.date(2022, 2, 2),
 datetime.date(2022, 2, 3),
 datetime.date(2022, 2, 4),
 datetime.date(2022, 2, 5),
 datetime.date(2022, 2, 6),
 datetime.date(2022, 2, 7),
 datetime.date(2022, 2, 8),
 datetime.date(2022, 2, 9),
 datetime.date(2022, 2, 10),
 datetime.date(2022, 2, 11),
 datetime.date(2022, 2, 12),
 datetime.date(2022, 2, 13),
 datetime.date(2022, 2, 14),
 datetime.date(2022, 2, 15),
 datetime.date(2022, 2, 16),
 datetime.date(2022, 2, 17),
 datetime.date(2022, 2, 18),
 datetime.date(2022, 2, 19),
 datetime.date(2022, 2, 20),
 datetime.date(2022, 2, 21),
 datetime.date(2022, 2, 22),
 datetime.date(2022, 2, 23),
 datetime.date(2022, 2, 24),
 datetime.date(2022, 2, 25),
 datetime.date(2022, 2, 26),
 datetime.date(2022, 2,

In [6]:
type_list = df["mediaType"].tolist()
uni_type_list = list(set(type_list))
uni_type_list

['Other', 'Reddit', 'Twitter', 'Forum', 'News', 'Blog']

In [7]:
for data_type in uni_type_list:
    print(data_type)
    target_df = df[df["mediaType"] == data_type]
    print(len(target_df))
    print(len(target_df)/1153528)

Other
3487
0.0030229001810099104
Reddit
6258
0.005425095879770582
Twitter
1032789
0.8953306725107669
Forum
42096
0.036493262408888213
News
50667
0.0439235111761483
Blog
18231
0.015804557843416025


In [8]:
lang_list = df["language"].tolist()
uni_lang_list = list(set(lang_list))
print(len(uni_lang_list))
uni_lang_list

74


['Russian',
 'Indonesian',
 'Cebuano',
 'Slovak',
 'Serbian',
 'Welsh',
 'Latvian',
 'Malagasy',
 'Belarusian',
 'French',
 'Marathi',
 'Albanian',
 'Romanian',
 'Sotho',
 'Kinyarwanda',
 'Irish',
 'Catalan',
 'Sundanese',
 'Turkish',
 'Korean',
 'Swedish',
 'Burmese',
 'Chinese',
 'Thai',
 'Chichewa',
 'Dutch',
 'Bulgarian',
 'German',
 'Uzbek',
 'Italian',
 'Vietnamese',
 'Icelandic',
 'Gujarti',
 'Arabic',
 'Azerbaijani',
 'Malayalam',
 'Czech',
 'Lithuanian',
 'Galician',
 'Portuguese',
 'Ganda',
 'Tagalog',
 'Polish',
 'Swahili',
 'Amharic',
 'Kannada',
 'Hebrew',
 'Georgian',
 'Javanese',
 'Gaelic',
 'Bosnian',
 'Estonian',
 'Malay',
 'Basque',
 'Japanese',
 'Haitian creole',
 'Armenian',
 'Norwegian',
 'Finnish',
 'Afrikaans',
 'Maltese',
 'Spanish',
 'Slovene',
 'Kurdish',
 'Tajik',
 'Urdu',
 'Greek',
 'Hungarian',
 'Danish',
 'Ukrainian',
 'Croatian',
 'Unknown',
 'Hindi',
 'English']

In [9]:
lang_dict = {}
for data_lang in uni_lang_list:
    target_df = df[df["language"] == data_lang]
    lang_dict[data_lang] = len(target_df) / 1153528
sort_lang_dict = {k: v for k, v in sorted(lang_dict.items(), key=lambda item: item[1])}
sort_lang_dict

{'Belarusian': 8.669057014654174e-07,
 'Marathi': 8.669057014654174e-07,
 'Burmese': 8.669057014654174e-07,
 'Malayalam': 8.669057014654174e-07,
 'Kannada': 8.669057014654174e-07,
 'Tajik': 8.669057014654174e-07,
 'Gujarti': 1.7338114029308349e-06,
 'Amharic': 1.7338114029308349e-06,
 'Cebuano': 2.6007171043962523e-06,
 'Georgian': 2.6007171043962523e-06,
 'Bulgarian': 3.4676228058616698e-06,
 'Urdu': 3.4676228058616698e-06,
 'Javanese': 4.334528507327087e-06,
 'Ganda': 6.068339910257922e-06,
 'Bosnian': 6.9352456117233395e-06,
 'Armenian': 6.9352456117233395e-06,
 'Croatian': 7.802151313188757e-06,
 'Gaelic': 8.669057014654174e-06,
 'Thai': 1.040286841758501e-05,
 'Chichewa': 1.1269774119050425e-05,
 'Kurdish': 1.3003585521981261e-05,
 'Maltese': 1.5604302626377515e-05,
 'Kinyarwanda': 1.7338114029308347e-05,
 'Icelandic': 1.7338114029308347e-05,
 'Hebrew': 1.7338114029308347e-05,
 'Afrikaans': 1.8205019730773767e-05,
 'Albanian': 2.080573683517002e-05,
 'Slovak': 2.6007171043962522e-

In [12]:
translated_df = df[df["translatedTitle"].notnull()]
print(len(translated_df))
translated_df

121175


,id,name,title,contentText,translatedTitle,translatedContentText,annotations,mediaType,mentionedUsers,author,mediaTypeAttributes,timePublished,language,embeddedUrls,imageUrls,datetime,date
1089969,79ef49b00a0305877328be30bbc0831491c5b3b9,messages/79ef49b00a0305877328be30bbc0831491c5b3b9,Les demandes d’asile retrouvent le niveau d’av...,Ce SUV de 2700 ch atteint une vitesse de point...,Asylum applications return to pre-pandemic level,This 2700 hp SUV reaches a top speed of 355 km...,"[{'id': '0d3db5ba95f248b0ff4c117c08ac7547', 'n...",News,[],msn.com,{},1642291200000,French,"[http://www.24heures.ch/, https://www.24heures...",[https://img-s-msn-com.akamaized.net/tenant/am...,2022-01-15 16:00:00,2022-01-15
1090431,7beb2f4e1ef79d4f82c586726a77e0b7cbecb01e,messages/7beb2f4e1ef79d4f82c586726a77e0b7cbecb01e,Le gouvernement du Bénin répond aux «pressions...,Interpellé mercredi 12 janvier par un député d...,The government of Benin responds to &quot;pres...,Questioned on Wednesday January 12 by a member...,"[{'id': 'fd0e556a27d67648b9a2b5634549569e', 'n...",News,[],RFI,{},1642264380000,French,[],[https://ic-cdn.flipboard.com/rfi.fr/e3ab4537d...,2022-01-15 08:33:00,2022-01-15
1048319,222d917f1bf3f666f2b31b005c2c83c9f0af2adc,messages/222d917f1bf3f666f2b31b005c2c83c9f0af2adc,Cyberattaque: l'Ukraine accuse la Russie et di...,Ajoute déclarations du porte-parole du Kremlin...,Cyberattack: Ukraine accuses Russia and says i...,"Adds statements by Kremlin spokesperson, Biden...","[{'id': 'c388a9f428bba9ee64f9a3856ce8bced', 'n...",News,[],zonebourse.com,{},1642350660000,French,[https://gateway.mdgms.com/extern/logo_image.h...,[https://gateway.mdgms.com/extern/logo_image.h...,2022-01-16 08:31:00,2022-01-16
1044040,1986efa834e0be219905cf4ceab726203b7dbe28,messages/1986efa834e0be219905cf4ceab726203b7dbe28,Plus de 360 millions de chrétiens persécutés d...,Plus de 360 millions de chrétiens persécutés d...,More than 360 million Christians persecuted in...,More than 360 million Christians persecuted in...,"[{'id': '0009421ef61214966aec82b64b2b5b89', 'n...",News,[],None,{},1642550400000,French,[https://portesouvertes.fr/persecution-des-chr...,[https://img.20mn.fr/uhmowooDRvCYTuLAUMjQSw/64...,2022-01-18 16:00:00,2022-01-18
1090797,7b615a2b60ab0973ee33ef4a78e7a40ed02f9bb3,messages/7b615a2b60ab0973ee33ef4a78e7a40ed02f9bb3,Actualités France :: France- Cameroun: CONFERE...,[FRANCE] [DIASPORA] L’Association Renaissance ...,News France :: France-Cameroon: PREPARATORY CO...,[FRANCE] [DIASPORA] The Renaissance Associatio...,"[{'id': '1a80b118b4aea4f430514cd02b9fc87f', 'n...",News,[],"Patrice EKWE EDIMO SILO, Président de l’Associ...",{},1642665480000,French,[],[https://www.camer.be/UserFiles/file/images/ca...,2022-01-19 23:58:00,2022-01-19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1073285,574d271950823eacc97136be0f8468ea4071e4fc,messages/574d271950823eacc97136be0f8468ea4071e4fc,Vers un suicide collectif de l’Europe?,Les vassaux doivent oser penser!\nDans le numé...,Towards a collective suicide of Europe?,Vassals must dare to think! In issue 6 of Hori...,"[{'id': '1a141897e43de88317cdd5b3d4750f15', 'n...",News,[],Silvia Cattori,{},1651324980000,French,[https://zeit-fragen.ch/fr/archives/2022/n-9-2...,[https://arretsurinfo.ch/wp-content/uploads/20...,2022-04-30 06:23:00,2022-04-30
1149629,f709fc61e91ff84656e775ae3d2722ea1e646f41,messages/f709fc61e91ff84656e775ae3d2722ea1e646f41,L'Ukraine dit que la Russie pilonne le Donbass...,KYIV (Reuters) - Les forces russes ont pilonné...,"Ukraine says Russia is pounding Donbass, unabl...",KYIV (Reuters) - Russian forces pounded the Do...,"[{'id': '1015032323cbdfd24263563d26030931', 'n...",News,[],zonebourse.com,{},1651317060000,French,[http://www.facebook.com/sharer.php?u=http://w...,[https://img.zonebourse.com/reuters/2022-04/20...,2022-04-30 04:11:00,2022-04-30
1077227,5f99f517738fa01347550e1bcee60728eccc3d7b,messages/5f99f517738fa01347550e1bcee60728eccc3d7b,Le HCR particulièrement inquiet du nom

In [10]:
tweet_df = df[df["mediaType"] == "Twitter"]
tweet_df

,id,name,title,contentText,translatedTitle,translatedContentText,annotations,mediaType,mentionedUsers,author,mediaTypeAttributes,timePublished,language,embeddedUrls,imageUrls,datetime,date
1146527,f1de26ea5b2a98440f0ea2e9cbb7e5975444396d,messages/f1de26ea5b2a98440f0ea2e9cbb7e5975444396d,RT @Fabien_Roussel J'en ai marre de cette gauc...,RT @Fabien_Roussel J'en ai marre de cette gauc...,RT @Fabien_Roussel I&#39;m fed up with this le...,RT @Fabien_Roussel I&#39;m fed up with this le...,"[{'id': 'ec5af9a56da3cb8193932709026e7fe1', 'n...",Twitter,[],None,"{'twitterData': {'retweetCount': 0, 'twitterAu...",1644911879000,French,[],[],2022-02-14 23:57:59,2022-02-14
1139822,e29f5b6b244b53fa4e44b978e44f2255eae9b5ca,messages/e29f5b6b244b53fa4e44b978e44f2255eae9b5ca,"RT @melenchon_2022 Avec @JLMelenchon, nous nou...","RT @melenchon_2022 Avec @JLMelenchon, nous nou...","RT @melenchon_2022 With @JLMelenchon, we are c...","RT @melenchon_2022 With @JLMelenchon, we are c...",[],Twitter,"[cb9738bb0d492549c2959afff3391cd5, 4eddbd184ea...",None,"{'twitterData': {'retweetCount': 0, 'twitterAu...",1644888920000,French,[],[https://video.twimg.com/ext_tw_video/14931509...,2022-02-14 17:35:20,2022-02-14
1061972,3e3cea1b2c68dc61d2c4241dee926b83ae72267d,messages/3e3cea1b2c68dc61d2c4241dee926b83ae72267d,Placer 1000000€ comme fond de réserve par cito...,Placer 1000000€ comme fond de réserve par cito...,"Place €1,000,000 as a reserve fund per citizen...","Place €1,000,000 as a reserve fund per citizen...",[],Twitter,[],None,"{'twitterData': {'retweetCount': 0, 'twitterAu...",1644911483000,French,[],[],2022-02-14 23:51:23,2022-02-14
1092589,7f4b798cb2c6561ab1cdd73a1d4f6b4edbe381d4,messages/7f4b798cb2c6561ab1cdd73a1d4f6b4edbe381d4,@vpecresse Les yeux dans les yeux je mens sans...,@vpecresse Les yeux dans les yeux je mens sans...,@vpecresse Eye to eye I lie without problem. D...,@vpecresse Eye to eye I lie without problem. D...,[],Twitter,[5166d3e4afe41e43c7556f000d370d9b],None,"{'twitterData': {'retweetCount': 0, 'twitterAu...",1644887986000,French,[],[],2022-02-14 17:19:46,2022-02-14
1099211,8d85edbe62983876b77927d00234585d42599144,messages/8d85edbe62983876b77927d00234585d42599144,RT @MLP_officiel Dans cette élection président...,RT @MLP_officiel Dans cette élection président...,RT @MLP_officiel In this presidential election...,RT @MLP_officiel In this presidential election...,"[{'id': 'e4802621028b95dbbb3a9d9aca9f9b75', 'n...",Twitter,[],None,"{'twitterData': {'retweetCount': 0, 'twitterAu...",1644910709000,French,[https://mlafrance.fr/],[http://pbs.twimg.com/media/FLkt791XIAIVxJZ.jpg],2022-02-14 23:38:29,2022-02-14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
243416,05c053a8a1341a0c467d6d0502523ee0255bbd5a,messages/05c053a8a1341a0c467d6d0502523ee0255bbd5a,RT @UPR_Asselineau 🤣 JE NE M'EN LASSE PAS...🤣\...,RT @UPR_Asselineau 🤣 JE NE M'EN LASSE PAS...🤣 ...,None,None,[],Twitter,[],None,"{'twitterData': {'retweetCount': 0, 'twitterAu...",1656596006000,French,[],[http://pbs.twimg.com/media/FWe0IncXEAE0rh9.jp...,2022-06-30 06:33:26,2022-06-30
117217,02c6c2907d1406cc178ee0354573fdbb0e9057a5,messages/02c6c2907d1406cc178ee0354573fdbb0e9057a5,RT @UPR_Asselineau 🤣 JE NE M'EN LASSE PAS...🤣\...,RT @UPR_Asselineau 🤣 JE NE M'EN LASSE PAS...🤣 ...,None,None,[],Twitter,[],None,"{'twitterData': {'retweetCount': 0, 'twitterAu...",1656621750000,French,[],[http://pbs.twimg.com/media/FWe0IncXEAE0rh9.jp...,2022-06-30 13:42:30,2022-06-30
626701,0ec9939120b893de4b07757516e9f42f49b02f78,messages/0ec9939120b893de4b07757516e9f42f49b02f78,RT @EmmanuelMacron La Russie ne doit pas l’emp...,RT @EmmanuelMacron La Russie ne doit pas l’emp...,None,None,[],Twitter,[],None,"{'twitterData': {'retweetCount': 0, 'twitterAu...",1656594931000,French,[],[https://video.twimg.com/amplify_video/1542481...,2022-06-30 06:15:31,2022-06-30
843002,13e59181f4bb89db955c21488ac6171d57f6cb56,messages/13e59181f4bb89db955c21488ac6171d57f6cb56,"RT @vpecresse À Kyiv, rencontre avec le pre

In [11]:
structural_features_raw = []
for row in tqdm(df.itertuples(), total=len(df)):
    twitter_data = row[11]
    twitter_features_dict = twitter_data['twitterData']
    # print(twitter_features_dict)
    structural_features_raw.append(twitter_features_dict)
    # break

structural_features_df = pd.DataFrame(structural_features_raw, index=df.index)
df = pd.concat([df, structural_features_df], axis=1)

  0%|          | 0/1153528 [00:00<?, ?it/s]


KeyError: 'twitterData'